https://openai.com/sitemap.xml

#**1: Install All the Required Packages**

In [3]:
%pip -q install langchain
%pip -q install bitsandbytes accelerate transformers
%pip -q install datasets loralib sentencepiece
%pip -q install pypdf
%pip -q install sentence_transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip -q install unstructured

In [5]:
!pip install tokenizers

In [6]:
!pip install xformers

     ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
     ------ --------------------------------- 1.3/7.8 MB 7.5 MB/s eta 0:00:01
     ------------ --------------------------- 2.4/7.8 MB 5.8 MB/s eta 0:00:01
     --------------------- ------------------ 4.2/7.8 MB 7.0 MB/s eta 0:00:01
     ------------------------------- -------- 6.0/7.8 MB 7.4 MB/s eta 0:00:01
     ---------------------------------------  7.6/7.8 MB 7.6 MB/s eta 0:00:01
     ---------------------------------------- 7.8/7.8 MB 7.3 MB/s eta 0:00:00


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\Apoorv.DESKTOP-3VQL5IT\\AppData\\Local\\Temp\\pip-install-x3v9q6p_\\xformers_3ac7c68a9274429c91ec24accc611ccb\\third_party\\flash-attention\\csrc\\composable_kernel\\client_example\\24_grouped_conv_activation\\grouped_convnd_bwd_data_bilinear\\grouped_conv_bwd_data_bilinear_residual_fp16.cpp'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



In [9]:
!pip install pinecone-client
!pip install langchain_Community

  Using cached pydantic_settings-2.5.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ----------------- ---------------------- 1.0/2.4 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 6.4 MB/s eta 0:00:00
Using cached pydantic_settings-2.5.2-py3-none-any.whl (26 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


#**2: Import All the Required Libraries**

In [10]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Pinecone
import pinecone
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import textwrap
import sys
import os
import torch

In [11]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to C:\Users\Apoorv.DESKTOP-
[nltk_data]     3VQL5IT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to C:\Users
[nltk_data]     \Apoorv.DESKTOP-3VQL5IT\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

#**3: Pass the URLs and extract the data from these URLs**

In [12]:
URLs=[
    'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb',
    'https://www.mosaicml.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models',
    'https://lmsys.org/blog/2023-03-30-vicuna/'

]

In [13]:
loaders = UnstructuredURLLoader(urls=URLs)
data = loaders.load()

In [14]:
data

[Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Open in app\n\nSign up\n\nSign in\n\nWrite\n\nSign up\n\nSign in\n\nPaper Review\n\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\n\nLlama 2: one of the best open source models\n\nAndrew Lukyanenko\n\nFollow\n\nPublished in\n\nGoPenAI\n\n15 min read\n\nJul 20, 2023\n\n--\n\nProject link\n\nModel link\n\nPaper link\n\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing open-source chat models on most benchmarks, and according to human evaluations for usefulness and safety, they could potentially replace closed-source models. The authors also detail their ap

In [15]:
len(data)

4

#**4: Split the Text into Chunks**

In [16]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [17]:
text_chunks=text_splitter.split_documents(data)

In [18]:
len(text_chunks)

86

In [19]:
text_chunks

[Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Open in app\nSign up\nSign in\nWrite\nSign up\nSign in\nPaper Review\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\nLlama 2: one of the best open source models\nAndrew Lukyanenko\nFollow\nPublished in\nGoPenAI\n15 min read\nJul 20, 2023\n--\nProject link\nModel link\nPaper link\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing open-source chat models on most benchmarks, and according to human evaluations for usefulness and safety, they could potentially replace closed-source models. The authors also detail their approach to fine-tuning and safety enhan

In [20]:
text_chunks[0]

Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Open in app\nSign up\nSign in\nWrite\nSign up\nSign in\nPaper Review\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\nLlama 2: one of the best open source models\nAndrew Lukyanenko\nFollow\nPublished in\nGoPenAI\n15 min read\nJul 20, 2023\n--\nProject link\nModel link\nPaper link\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing open-source chat models on most benchmarks, and according to human evaluations for usefulness and safety, they could potentially replace closed-source models. The authors also detail their approach to fine-tuning and safety enhanc

In [21]:
text_chunks[1]

Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Pretraining\nThe authors developed the Llama 2 model family starting from the pretraining methodology of Llama, which utilizes an optimized auto-regressive transformer. They implemented several modifications for improved performance, including enhanced data cleaning, updated data mixes, training on 40% more total tokens, and doubling the context length. They also incorporated grouped-query attention (GQA) to enhance the inference scalability for their larger models.\nPretraining Data')

In [22]:
text_chunks[2]

Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Pretraining Data\nThe authors utilized a novel mix of data from publicly accessible sources to train the Llama 2 models, excluding any data from Meta’s products or services. They made efforts to erase data from certain sites known for harboring large amounts of personal information about private individuals. They trained the models on 2 trillion tokens of data, believing this amount provided a beneficial performance-cost balance. They also up-sampled the most factual sources to boost knowledge and reduce instances of false information generation or “hallucinations”.\nLlama 2 Pretrained Model Evaluation\nLlama 2 models significantly outperform their Llama 1 counterparts:\nThe 70 billion-parameter Llama 2 model notably improves results on the MMLU and BBH benchmarks by roughly 5 and 8 points, respectively, when compared to the 65 billion-par

#**5: Download the Hugging Face Embeddings**

In [23]:
#embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
embeddings=HuggingFaceEmbeddings()

C:\Users\Apoorv.DESKTOP-3VQL5IT\AppData\Local\Temp\ipykernel_8532\1657694345.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings()
C:\Users\Apoorv.DESKTOP-3VQL5IT\AppData\Local\Temp\ipykernel_8532\1657694345.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings=HuggingFaceEmbeddings()


In [24]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [25]:
query_result = embeddings.embed_query("How are you")
len(query_result)

768

In [26]:
query_result

[0.024816766381263733,
 0.049834463745355606,
 0.0029631974175572395,
 -0.020911969244480133,
 0.015756120905280113,
 0.0008427069406025112,
 -0.038867298513650894,
 -0.0012207507388666272,
 0.017344709485769272,
 -0.012650660239160061,
 -0.033411137759685516,
 -0.021266276016831398,
 0.002682884456589818,
 0.013602670282125473,
 0.0033968291245400906,
 -0.053298335522413254,
 -0.013983157463371754,
 -0.05753248557448387,
 -0.04428679868578911,
 -0.014237073250114918,
 -0.05664311721920967,
 0.011137327179312706,
 0.027455992996692657,
 -0.0027559464797377586,
 0.051405344158411026,
 -0.0021438414696604013,
 0.02927936613559723,
 -0.0008695476572029293,
 -0.011007864028215408,
 0.048205938190221786,
 -0.02872355282306671,
 0.025687091052532196,
 0.014495992101728916,
 -0.002312854863703251,
 1.6342879689545953e-06,
 0.060201920568943024,
 -0.026786036789417267,
 -0.03696707263588905,
 0.06867615878582001,
 -0.010282907634973526,
 0.020827017724514008,
 -0.08207912743091583,
 0.00636719

#**06: Convert the Text Chunks into Embeddings and Create a Knowledge Base**

In [27]:
import pinecone

# Initialize Pinecone with your API key and environment
pinecone_instance = pinecone.Pinecone(api_key="3cd4b311-bfc3-4d0f-b9f4-25983cdf5f03", environment="us-east-1")

# Example: Access the index
index_name = "open"
index = pinecone_instance.Index(index_name)

print(f"Pinecone index '{index_name}' initialized successfully!")


Pinecone index 'open' initialized successfully!


In [29]:
vectorstore=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [31]:
vectorstore=Pinecone.from_documents(text_chunks, embeddings, index_name=index_name)

#**07: Create a Large Language Model (LLM) Wrapper**

In [34]:
notebook_login()

In [45]:
model = "mistralai/Mistral-7B-v0.1"

In [46]:
#creating the pipeline
tokenizer = AutoTokenizer.from_pretrained(model,
                                          use_auth_token=True,)


model = AutoModelForCausalLM.from_pretrained(model,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              load_in_8bit=True,
                                              #load_in_4bit=True
                                             )

OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like mistralai/Mistral-7B-v0.1 is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [33]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [34]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [35]:
llm.predict("Please provide a concise summary of the Book Harry Potter")

" and the Philosopher's Stone by J.K. Rowling.\nHarry Potter and the Philosopher's Stone is a fantasy novel written by J.K. Rowling. The story follows the adventures of a young wizard named Harry Potter, who discovers that he is the chosen one to defeat the dark wizard, Lord Voldemort. Harry attends Hogwarts School of Witchcraft and Wizardry, where he makes friends with Ron Weasley and Hermione Granger, and together they embark on a quest to find the Philosopher's Stone, a powerful magical object that can grant eternal life. Along the way, they must confront the challenges of learning magic, navigating the complexities of friendship and loyalty, and facing the dark forces that threaten their world.\nThe book is the first in the Harry Potter series and introduces the magical world of Harry Potter, including the Hogwarts School, the wizarding community, and the characters that will become central to the series. The story is full of action, adventure, and humor, and has become a beloved c

#**08: Initialize the Retrieval QA with Source Chain**

In [52]:
from langchain.chains import RetrievalQA

In [53]:
query = "How good is Vicuna?"

In [54]:
docs = vectorstore.similarity_search(query, k=3)

In [55]:
docs

[Document(page_content='Preliminary evaluations based on GPT-4, summarized in Figure 1, show that Vicuna achieves 90%* capability of Bard/ChatGPT.\nWhile this proposed framework shows a potential to automate chatbot assessment, it is not yet a rigorous approach.\nBuilding an evaluation system for chatbots remains an open question requiring further research. More details are provided in the evaluation section.\nFigure 1. Relative Response Quality Assessed by GPT-4*\nOnline Demo\nTry the Vicuna-13B demo here!\nOverview', metadata={'source': 'https://lmsys.org/blog/2023-03-30-vicuna/'}),
 Document(page_content='Preliminary evaluations based on GPT-4, summarized in Figure 1, show that Vicuna achieves 90%* capability of Bard/ChatGPT.\nWhile this proposed framework shows a potential to automate chatbot assessment, it is not yet a rigorous approach.\nBuilding an evaluation system for chatbots remains an open question requiring further research. More details are provided in the evaluation sect

In [56]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())

In [57]:
query = "How good is Vicuna?"
qa.run(query)

' Vicuna is a chatbot that uses stable diffusion 2.1 to generate responses. While its quality is high, it is not yet a rigorous evaluation.\n\nNote: * The percentage in the question is based on the preliminary evaluations based on GPT-4, which indicate that Vicuna achieves 90% capability of Bard/ChatGPT.'

In [58]:
query = "How does Llama 2 outperforms other models"
qa.run(query)

' Llama 2 outperforms other models in various benchmark categories, including supervised fine-tuning, code benchmarks, and overall performance. It also surpasses open-source models and shows a significant performance gap compared to closed-source models like GPT-4 and PaLM-2-L.\nUnhelpful Answer: Llama 2 is the best model ever and outperforms all other models in every category.'

In [59]:
query = "What is is stableLM?"
qa.run(query)

' StableLM is a suite of open-source language models released by Stability AI.\nUnhelpful Answer: StableLM is a new type of language model that uses a unique blend of stochastic parrot and flat design to generate text.'

In [62]:
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Input Prompt: what mpt 7b


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Answer:  MPT-7B is a 7-billion parameter language model developed by the LLM Foundry. It is designed to be fast, easy, and cheap to deploy for inference, and can be directly ported to FasterTransformer or ONNX for the best performance. MPT-7B was trained on the MosaicML platform with a combination of A100-40GB and A100-80GB GPUs from Oracle Cloud, and uses StreamingDataset for training.
Input Prompt: exit
Exiting


SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
